# Load Colab

In [1]:
# Please select "GPU" mode

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Change to current directory

In [2]:
import os
os.chdir('./drive/MyDrive/Demand-prediction')

# Import packages

In [3]:
import pandas as pd  
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F


In [4]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

In [5]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def compute_metric(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = mean_absolute_percentage_error(y_true[np.where(y_true > 5)[0]], y_pred[np.where(y_true > 5)[0]])
    return mae, rmse, mape

In [6]:
def get_dataloader(X, y, device, bs, shuffle):
    return torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.FloatTensor(X).to(device), 
                    torch.FloatTensor(y).to(device)), batch_size=bs, shuffle=shuffle, drop_last=False)

def calculate_metric_torch(true, pred, mask_value=5):
    mae = torch.mean(torch.abs(true - pred))
    rmse = torch.sqrt(torch.mean((pred - true) ** 2))
    if mask_value != None:
        mask = torch.gt(true, mask_value)
        pred = torch.masked_select(pred, mask)
        true = torch.masked_select(true, mask)
    mape = torch.mean(torch.abs(torch.div((true - pred), true)))
    return mae, rmse, mape

def trainer(model, lr, epochs, train_loader, test_loader):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    MSE = nn.MSELoss()
    
    for epoch in range(epochs):
        model.train()
        epoch_loss = list()
        for src, trg in train_loader:
            pred = model(src)
            loss = MSE(pred, trg)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
            epoch_loss.append(loss.item())
        epoch_loss = np.mean(epoch_loss)
        
        model.eval()
        with torch.no_grad():
            preds, trues = list(), list()
            for src, trg in test_loader:
                pred = model(src)
                preds.append(pred)
                trues.append(trg)
            
            mae, rmse, mape = calculate_metric_torch(torch.cat(trues), torch.cat(preds))
        
        print('Epoch %d, training loss: %.3f, test mae: %.3f, rmse: %.3f, mape: %.3f' % (epoch, epoch_loss, mae, rmse, mape))


# Load the dataset

In [12]:
datasets = np.load('./data/processed_demand_datasetsMAN.npz')
X_train, X_val, X_test, y_train, y_val, y_test = datasets['trainX'], datasets['valX'], datasets['testX'], datasets['trainy'], datasets['valy'], datasets['testy']

# In oder to accelearte the training process, we downsample the dataset (1/12)
X_train = X_train[:,::12,:]
X_val = X_val[:,::12,:]
X_test = X_test[:,::12,:]
y_train = y_train[:,::12]
y_val = y_val[:,::12,]
y_test = y_test[:,::12]

num_nodes = X_train.shape[0]
edges = np.load('./data/edges_GAman.npy')

adj = np.zeros((num_nodes, num_nodes))

for i in range(num_nodes): 
    adj[i,i] = 1
    
for i in range(len(edges)):
    adj[edges[i,0], edges[i,1]] = 1
    
adj = torch.LongTensor(adj)

device = torch.device('cuda')

# RF

In [23]:
rf = RandomForestRegressor(random_state=1000)
rf.fit(X_train.reshape(-1,42), y_train.reshape(-1))

pred = rf.predict(X_test.reshape(-1,42))
mae, rmse, mape = compute_metric(y_test.reshape(-1), pred)
print('test mae: %.3f, rmse: %.3f, mape: %.3f' % (mae, rmse, mape))


test mae: 2.146, rmse: 4.371, mape: 21.089


# GBDT

In [24]:
gbdt = GradientBoostingRegressor(random_state=1000)
gbdt.fit(X_train.reshape(-1,42), y_train.reshape(-1))

pred = gbdt.predict(X_test.reshape(-1,42))
mae, rmse, mape = compute_metric(y_test.reshape(-1), pred)
print('test mae: %.3f, rmse: %.3f, mape: %.3f' % (mae, rmse, mape))

test mae: 2.183, rmse: 4.371, mape: 21.174


# MLP

In [25]:
mlp = MLPRegressor(random_state=200)
mlp.fit(X_train.reshape(-1,42), y_train.reshape(-1))

pred = mlp.predict(X_test.reshape(-1,42))
mae, rmse, mape = compute_metric(y_test.reshape(-1), pred)
print('test mae: %.3f, rmse: %.3f, mape: %.3f' % (mae, rmse, mape))

test mae: 2.139, rmse: 4.328, mape: 21.091


# GRU

In [16]:
class GRU(nn.Module):
    def __init__(self, hidden_dim1, num_layers, hidden_dim2):
        super().__init__()
        self.GRU = nn.GRU(1, hidden_dim1, num_layers, batch_first=True)
        self.Dense = nn.Sequential(nn.Linear(hidden_dim1, hidden_dim2), nn.ReLU(), nn.Linear(hidden_dim2, 1))
    
    def forward(self, x):
        o, h = self.GRU(x)
        x = self.Dense(h[-1])
        return x.flatten()

In [18]:
train_loader = get_dataloader(X_train.reshape(-1,42,1), y_train.reshape(-1), device, 64, True)
test_loader = get_dataloader(X_test.reshape(-1,42,1), y_test.reshape(-1), device, 64, False)

lr = 0.001
epochs = 20

model = GRU(64, 2, 32).to(device)

trainer(model, lr, epochs, train_loader, test_loader)


Epoch 0, training loss: 90.566, test mae: 2.311, rmse: 4.624, mape: 0.241
Epoch 1, training loss: 25.840, test mae: 2.186, rmse: 4.500, mape: 0.214
Epoch 2, training loss: 25.149, test mae: 2.421, rmse: 4.607, mape: 0.242
Epoch 3, training loss: 24.829, test mae: 2.271, rmse: 4.806, mape: 0.219
Epoch 4, training loss: 24.261, test mae: 2.147, rmse: 4.371, mape: 0.216
Epoch 5, training loss: 24.306, test mae: 2.203, rmse: 4.398, mape: 0.213
Epoch 6, training loss: 24.045, test mae: 2.147, rmse: 4.425, mape: 0.213
Epoch 7, training loss: 23.708, test mae: 2.163, rmse: 4.374, mape: 0.216
Epoch 8, training loss: 23.646, test mae: 2.188, rmse: 4.517, mape: 0.217
Epoch 9, training loss: 23.544, test mae: 2.192, rmse: 4.427, mape: 0.213
Epoch 10, training loss: 23.109, test mae: 2.146, rmse: 4.456, mape: 0.210
Epoch 11, training loss: 23.060, test mae: 2.229, rmse: 4.409, mape: 0.221
Epoch 12, training loss: 22.975, test mae: 2.164, rmse: 4.464, mape: 0.218
Epoch 13, training loss: 23.067, te

# GRU-GAT

In [19]:
class GATLayer(nn.Module):
    def __init__(self, device, in_dim, out_dim, alpha=0.2, dropout=0.5):
        super().__init__()
        self.device = device
        self.alpha = alpha
        self.dropout = dropout
        self.weights = nn.Parameter(torch.FloatTensor(in_dim, out_dim))
        nn.init.xavier_normal_(self.weights.data, gain=1.414)
        self.a = nn.Parameter(torch.FloatTensor(2*out_dim, 1))
        nn.init.xavier_normal_(self.a.data, gain=1.414)

    def forward(self, h, adj):
        """
        h: (bs, num_node, in_dim)
        adj: (num_node, num_node)
        return (bs, num_node, out_dim)
        """
        bs, num_node, in_dim = h.size()
        src, trg = torch.nonzero(adj.long(), as_tuple=True)
        
        Wh = torch.matmul(h, self.weights) # (bs, num_node, out_dim)
        edge_h = torch.cat([Wh[:,src,:], Wh[:,trg,:]], dim=-1) # (bs, num_edges, 2*out_dim)
        edge_e = F.leaky_relu(torch.matmul(edge_h, self.a), negative_slope=self.alpha).squeeze(-1) # (bs, num_edges)

        attention = -9e15*torch.ones(bs, num_node, num_node).to(self.device) #（bs, num_node, num_node)
        attention[:, src, trg] = edge_e
        attention = F.dropout(F.softmax(attention, dim=-1), self.dropout) #（bs, num_node, num_node)

        h_prime = torch.einsum('bij,bjo->bio', attention, Wh)

        return h_prime
    
    
class GRUGAT(nn.Module):
    def __init__(self, device, in_dim, out_dim, num_node, adj):
        super().__init__()
        self.adj = adj
        self.gru = nn.GRU(1, in_dim, 2, batch_first=True)
        self.gat = GATLayer(device, in_dim, out_dim)
        self.end_conv = nn.Conv1d(out_dim, 1, kernel_size=1)
        
    def forward(self, x):
        # x: (bs, num_node, time_step)
        X = []
        for i in range(x.size(1)):
            o, h = self.gru(x[:,i,:].unsqueeze(-1))
            X.append(h[-1])
        X = torch.stack(X)
        X = self.gat(X.permute(1,0,2), self.adj) # (bs, num_node, 16)
        X = self.end_conv(X.permute(0,2,1))
        return X.squeeze(1)
    

In [22]:
train_loader = get_dataloader(X_train.transpose(1,0,2), y_train.T, device, 36, True)
test_loader = get_dataloader(X_test.transpose(1,0,2), y_test.T, device, 36, False)

lr = 0.001
epochs = 10

model = GRUGAT(device, 10, 32, 198, adj).to(device)

trainer(model, lr, epochs, train_loader, test_loader)


Epoch 0, training loss: 770.729, test mae: 11.494, rmse: 24.313, mape: 0.860
Epoch 1, training loss: 714.988, test mae: 12.339, rmse: 23.313, mape: 0.699
Epoch 2, training loss: 664.571, test mae: 11.232, rmse: 22.020, mape: 0.673
Epoch 3, training loss: 596.089, test mae: 11.065, rmse: 20.938, mape: 0.664
Epoch 4, training loss: 542.924, test mae: 10.805, rmse: 20.197, mape: 0.632
Epoch 5, training loss: 514.201, test mae: 10.589, rmse: 19.616, mape: 0.645
Epoch 6, training loss: 492.659, test mae: 10.476, rmse: 19.498, mape: 0.639
Epoch 7, training loss: 487.002, test mae: 10.606, rmse: 19.652, mape: 0.659
Epoch 8, training loss: 478.232, test mae: 10.598, rmse: 19.417, mape: 0.657
Epoch 9, training loss: 472.627, test mae: 10.591, rmse: 19.316, mape: 0.650
